# Context Free Grammar Parsing using the CYK Algorithm
## Muya Guoji and Evelyn Kessler | December 17, 2023

### Table of Contents
1. Introduction
    1. What is CFG Parsing?
    2. CFG Parsing in the Real World
2. The CYK Algorithm
    1. Chomsky Normal Form
    2. Converting CFGs into Chomsky Normal Form (CNF)
    3. Basic Code Architecture
    4. Parsing a Valid Example String (Visual Walkthrough)
    5. Parsing an Invalid Example String (Visual Walkthrough)
3. Code Walkthrough
    1. The 'CYK_parse' Function
    2. The 'print_table' Function - note: rewrite
    3. Testing with Examples

4. CYK Parsing on Natural Langauge
    1. The Benefits of CYK in NLP: a Brief Comparative Analysis with Transformers
    2. What are Parts of Speech?
    3. Writing CNF CFGs for Natural Language Parsing
    4. Parsing a Sentence (Visual Walkthrough)
5. Appendix
    1. Instructions for Running the Algorithm
    2. CYK Implementation with Examples
6. Sources

### Introduction
A context free grammar, or CFG, is a grammar that is used to generate all possible strings in a given language. A context free grammar, G, can be defined by a tuple 

$$
G = (\Sigma, \Gamma, R, S)
$$

where $\Sigma$ is a list of terminal symbols, $\Gamma$ is a list of non-terminal symbols, R is a list of rules of the form $x \rightarrow \mu$ where $\mu \in (\Gamma \cup \Sigma)*$, and $S$ is the start symbol. One way of defining a language is to write a CFG for that language. Then, we can find for any string whether that string is in the language by evaluating whether the rules of the CFG could possibly generate that string. For this method we start with the start symbol $S$ and apply the rules continuously until we generate the desired string. If we can't generate the string after (a reasonable number of) rules applications *double check this is actually right; are CFGs where you get stuck or where you run out??*, we can conclude that the given string is not valid for the language described by that CFG.

#### What is CFG Parsing?
CFG parsing is a method used to determine if a given string conforms to the rules of a specific language as defined by its CFG. Unlike the previous CFG method, which begins with the start symbol and attempts to construct the string by applying grammar rules, CFG parsing starts with the given string and works backwards, trying to decompose it into its basic elements until it reaches the start symbol. This process involves applying the CFG rules in reverse.

CFG parsing can be implemented with several different strategies and algorithms. These include top-down parsing methods like recursive descent parsing, where the parser starts at the highest level of the grammar and works its way down, and bottom-up methods like shift-reduce parsing, which start with the input and gradually transform it into the syntax tree of the grammar.

The essential part of CFG parsing is the construction of a parse tree or syntax tree. This tree represents the syntactic structure of the string according to the grammar rules. Each node in the tree corresponds to a grammar rule, and the leaves represent the elements of the string. By analyzing the tree, we can understand how the string is structured according to the grammar and whether it's a valid construct in the language.

CFG parsing is crucial in various applications because it allows for the automatic analysis of the structure of strings, which is fundamental in language processing, programming languages, and other computational linguistics areas. It enables computers to understand and manipulate complex structures in human and programming languages, making it a cornerstone of modern computing.

#### CFG Parsing in the Real World
CFG parsing is valuable in many practical scenarios, particularly in computer science and linguistics. A prime example is in natural language processing (NLP), where CFG parsing helps computers make sense of human language. It breaks down sentences into grammatical parts, which is crucial for things like translating languages since computers need to get the structure right to translate words accurately from one language to another. 

Another key use of CFG parsing in the development of compilers and interpreters for programming languages. The ability to parse context-free grammars is useful for syntax analysis, ensuring that the code written by programmers adheres to the grammatical rules of the programming language. This type or parsing is vital for error detection and correction during software development. 

Additionally, in more specialized areas, the CFG parsing can be applied in DNA sequencing and bioinformatics for analyzing and interpreting the genetic information contained in DNA sequences, where it helps in understanding the complex structures and patterns within genetic data. 

Besides these, CFG parsing finds its place in areas such as helping voice recognition systems understand what we say and analyzing complex mathematical formulas in scientific programs.


### The CYK Algorithm

#### Chomsky Normal Form
Before delving into the algorithm, we would like to first touch on the concept of Chomsky Normal Form (CNF). It is a prerequisite for understanding the CYK algorithm, as the algorithm specifically requires grammars to be in CNF in order to be processed. 

CNF, developed by the well-known modern linguist Noam Chomsky, aims to simplify the rules of context-free
grammars for more efficient parsing and algorithmic analysis. It is a specific method for expressing CFGs, wherein every production rule takes one of two forms: either a non-terminal symbol producing two other non-terminal symbols or a non-terminal symbol producing a single terminal symbol. This single terminal symbol may include epsilon, indicating the deletion of a sentence. 

$$
A \rightarrow BC
$$
$$
A \rightarrow a
$$
$$
S \rightarrow \Epsilon
$$
*(Uppercase letters represent non-terminal symbols, and lowercase letters represent terminal symbols)*


By unifying production rules in such a specific and clean form really simplifies the design and analysis of parsing
algorithms like CYK. 

#### Converting CFGs into Chomsky Normal Form (CNF)

In CNF, each rule must either produce two non-terminal symbols or a single terminal symbol. Consider the grammars below.

$$G1 = S \rightarrow a, S \rightarrow AZ, A \rightarrow a, Z \rightarrow z$$
$$G2 = S \rightarrow a, S \rightarrow aZ, Z \rightarrow a$$

Grammar $G1$ is in CNF since the first, third, and fourth rules all produce a single terminal symbol and the second rule produces two non-terminal symbols. Grammar $G2$ is not in CNF since the rule $S \rightarrow aZ$ contains a terminal symbol ($a$) followed by a non-terminal symbol ($Z$).

To convert a CFG to CNF, follow these rules:

1. **Eliminate the start symbol from right hand side (RHS)**: If there is a start symbol S in the RHS of any rule, create a new rule $S0 \rightarrow S$ and replace the $S$ in the RHS with $S0$.

2. **Eliminate Null Productions**: Remove any rules that produce an empty string, replacing them with alternative productions that achieve the same language representation without the null option.

3. **Eliminate Unit Productions**: Replace rules that produce a single non-terminal with rules that produce terminals directly. For instance $A \rightarrow B$ and $B \rightarrow a$ can be replaced with $A \rightarrow a$.

4. **Ensure all rules with terminals in RHS have a single terminal**: Eliminate terminals from the RHS if they are with other terminals or non-terminals by decomposing into multiple rules. For example, if you have the rule $X \rightarrow xY$ you can decompose it into $X \rightarrow ZY$ and $Z \rightarrow x$.

5. **Ensure all rules with non-terminals in RHS have exactly two non-terminals** : Eliminate rules with more than two non-terminal symbols by decomposing them into multiple rules. For example, the rule $X \rightarrow XYZ$ can be broken into $X \rightarrow PZ$ and $P \rightarrow XY$.

Let's try an example. The grammar $S \rightarrow \epsilon, S \rightarrow aSb$ generates the language $a^nb^n | n \geq 0$.

1. Eliminate start symbols from the right hand side.
We can eliminate the S from the RHS by writing a new rule $S0 \rightarrow S$ where S0 is our new start symbol.

$$S \rightarrow \epsilon$$
$$S \rightarrow aSb$$
$$S0 \rightarrow S$$

2. Eliminate Null Productions
Let's remove the epsilon and add a rule to cover the case where the $S \rightarrow \epsilon$ would be used. If this rule were used, the $\epsilon$ would replace the $T$ in $S \rightarrow aTb$ leaving just the $a$ and $b$. We can replicate this behavior by declaring $S \rightarrow ab$. Note that this does not cover the case where $n=0$. To cover this case, we need to add back in a rule $S \rightarrow \epsilon$.

$$S \rightarrow ab$$
$$S \rightarrow aSb$$
$$S \rightarrow \epsilon$$
$$S0 \rightarrow S$$

3. Eliminate Unit Productions
We have one unit production in our grammar, $S0 \rightarrow S$. We can fix this by explicitely defining what $S0$ can go to as $S0 \rightarrow ab | aSb | \epsilon$. Note that $S0$ can go to $\epsilon$ to create the empty string, but $S$ no longer goes to $\epsilon$ since we've replaced that case.

$$S \rightarrow ab | aSb$$
$$S0 \rightarrow ab | aSb | \epsilon$$

4. Ensure all rules with terminals in RHS have a single terminal
The rules $S \rightarrow aSb$ and $S \rightarrow ab$ have multiple terminals and/or terminals with non-terminals, as do their $S0$ equivalents. We can decompose them by introducing new non-terminals, $A \rightarrow a$ and $B \rightarrow b$.

$$S \rightarrow AB | ASB$$
$$S0 \rightarrow AB | ASB | \epsilon$$
$$A \rightarrow a$$
$$B \rightarrow b$$

5. Ensure all rules with non-terminals in RHS have exactly two non-terminals
The rules $S \rightarrow ASB$ and $S0 \rightarrow ASB$ have three non-terminals. We can introduce a new non-terminal to split the rule into $S \rightarrow AR$ and $R \rightarrow SB$ and same for $S0$.

$$S \rightarrow AB | AR$$
$$S0 \rightarrow AB | AR | \epsilon$$
$$R \rightarrow SB$$
$$A \rightarrow a$$
$$B \rightarrow b$$

After covering the five steps, we've converted our CFG into CNF! Note that this is not the only possible CNF for this language, there are many options that will work.
Let's double check that our CNF CFG makes sense for the language $a^nb^n | n \geq 0$ using our parser! (Note that we renamed some non-terminals for ease of use in the algorithm. S0 is renamed S and S is renamed T).

In [5]:
from CYKalgorithm import CYK_parse

grammar_anbn = {
    'S': {'AB', 'AR', 'e'}, #S: start sym. note that we've replaced S0 with S and S with T.
    'T': {'AB', 'AR'},
    'R': {'TB'},
    'A': {'a'},
    'B': {'b'}
}

# Example strings (TRUE)
string_anbn = "aabb"
string1_abnb = ""
string2_anbn = "ab"

# Example strings (FALSE)
string3_anbn = "abbb"
string4_anbn = "b"
string5_anbn = "abab"

table, result = CYK_parse(grammar_anbn, string_anbn) #change the string entry to other example string#_anbn to see different results, or add your own string!
print("Can the string be derived?", result)

#Don't worry about understanding the parsing table being printed just yet. For a full walkthrough of the parsing table, see *Visual Walkthrough* below.

a found in ('A', {'a'}). Adding A to T[0][0]
Parsing table:
[{'A'}, set(), set(), set()]
[set(), set(), set(), set()]
[set(), set(), set(), set()]
[set(), set(), set(), set()]
a found in ('A', {'a'}). Adding A to T[1][1]
Parsing table:
[{'A'}, set(), set(), set()]
[set(), {'A'}, set(), set()]
[set(), set(), set(), set()]
[set(), set(), set(), set()]
b found in ('B', {'b'}). Adding B to T[2][2]
Parsing table:
[{'A'}, set(), set(), set()]
[set(), {'A'}, set(), set()]
[set(), set(), {'B'}, set()]
[set(), set(), set(), set()]
b found in ('B', {'b'}). Adding B to T[3][3]
Parsing table:
[{'A'}, set(), set(), set()]
[set(), {'A'}, set(), set()]
[set(), set(), {'B'}, set()]
[set(), set(), set(), {'B'}]
A found in T[1][1] and B found in T[2][2]. S : AB is valid. Adding S to T[1][2]
Parsing table:
[{'A'}, set(), set(), set()]
[set(), {'A'}, {'S'}, set()]
[set(), set(), {'B'}, set()]
[set(), set(), set(), {'B'}]
A found in T[1][1] and B found in T[2][2]. T : AB is valid. Adding T to T[1][2]
Parsi

#### Basic Code Architecture
Let's begin the code run-through of the algorithm with a high-level overview of the CYK algorithm's architecture:

1. **Input and Grammar Preparation**: Before algorithm starts, the model implementer should prepare with an input string, which are sentences breaking down into words, and a set of grammar rules in Chomsky Normal Form (CNF).

2. **Table Initialization**: A table (matrix) is then created with dimensions based on the length of the input string. This table will be used to store possible grammar derivations for substrings of the input.

3. **Table Filling**: starting from the bottom level, the algorithm fills in the table and moves upward gradually. Each cell in the table represents a substring of the sentence and is filled with all possible grammar rules that could generate that substring. 

4. **Combining Substrings**: As it moves up the table, the algorithm combines smaller substrings that have already been matched with grammar rules to form larger substrings. It checks to see if these larger substrings can be generated by any of the CFG rules.

5. **Final Verification**: Once the table is filled, the algorithm checks the top cell, which represents the entire string (sentence). If this cell contains the start symbol of the grammar, the string is considered derivable from the given grammar.

6. **Result and Visualization**: Visualization of the table after each iteration can be used to enhance clarity and explainability of the processes for model implementers.

#### Parsing a Valid Example String (Visual Walkthrough)

Let's do a quick walkthrough of the algorithm on the CFG for the language a^nb^n | n >= 0. In this walkthrough we will explain how the parsing table is generated for the string "aabb" on the grammar:

$$S \rightarrow AB | AR | \epsilon$$
$$T \rightarrow AB | AR$$
$$R \rightarrow TB$$
$$A \rightarrow a$$
$$B \rightarrow b$$

where S is the start symbol and epsilon is the empty symbol.

1. Create an empty table.

Because the string has a length of four characters, our table will be of size 4 x 4. Note the black boxes in the table. Since we are filling the table by combining lower cells, we will not be using the black boxes.

![Empty Table](walkthrough1.jpg)

2. Fill the bottom row with the terminal to non-terminal rules.

We have two rules to convert terminals (a, b) into non-terminals. These are $A \rightarrow a$ and $B \rightarrow b$.

![First Row](walkthrough2.jpg)

3. Fill the next row by finding valid combinations of neighboring boxes.

In order to fill the next row, we need to see if any combinations of neighboring boxes yield the RHS of one of our grammar rules. The only combination in this table is the combination AB for the rules $S \rightarrow AB$ and $T \rightarrow AB$. We can fill the box above the combination AB with the LHS of the rules, S and T.

![Second Row](walkthrough3.jpg)

4. Fill next row by finding valid combinations of "top level" boxes.

From now on, we will look at either neighboring or "top level" boxes. Top level boxes are boxes that have only spaces above them (no other filled boxes or black boxes). In the previous table, the top level boxes are A, {S, T}, and B. Disregarding spaces, we once again look for neighbors among the top level boxes. This means we are looking at the combinations AS, AT, SB, and TB. Of those, only TB is a valid combination for the rule $R \rightarrow TB$. So we fill the box above the combination TB with R.

![Third Row](walkthrough4.jpg)

5. Fill the final row with valid combinations of "top level" boxes.

We repeat this process to fill the final row. The top level boxes are A, R, and B. The combination AR is valid for the rules $S \rightarrow AR$ and $T \rightarrow AR$, so we can fill the box above the combination AR with S and T. Note that when we fill the box above a combination, we always fill the box in the same column as the left piece of the combination (in this case A) and the 
row above the highest piece of the combination (in this case R). If it helps, you can shift the boxes in the chart (either in your mind or when you draw your own charts) and imagine them as a centered pyramid.

![Final Row](walkthrough5.jpg)

6. Check if the top-left box contains the start symbol, S.

If the string is valid for the grammar, then the top-left box will contain the start symbol, S. Since our top-left box contains S, we can conclude that 'aabb' is a valid string for the CFG, and therefore is within the language defined by the CFG, a^nb^n for any n >= 0.

#### Parsing an Invalid Example String (Visual Walkthrough)
Let's use the same grammar, but this time try and parse the string 'abab'. We know from the definition of the language that this string is not part of the language, i.e. it doesn't have n a's followed by n b's. 

1. Create an empty table.

Because the string has a length of four characters, our table will be of size 4 x 4.

![Empty Table](walkthrough1_not.jpg)

2. Fill the bottom row with the terminal to non-terminal rules.

We have two rules to convert terminals (a, b) into non-terminals. These are $A \rightarrow a$ and $B \rightarrow b$.

![First Row](walkthrough2_not.jpg)

3. Fill the next row by finding valid combinations of neighboring boxes.

In order to fill the next row, we need to see if any combinations of neighboring boxes yield the RHS of one of our grammar rules. The only combination in this table is the combination AB for the rules $S \rightarrow AB$ and $T \rightarrow AB$. This combination is present twice. We can fill the boxes above the combinations AB with the LHS of the rules, S and T.

![Second Row](walkthrough3_not.jpg)

4. Fill next row by finding valid combinations of "top level" boxes.

In the previous table, the top level boxes are {S, T} and B. Disregarding spaces, we once again look for neighbors among the top level boxes. This means we are looking at the combinations SB and TB. Of those, only TB is a valid combination for the rule $R \rightarrow TB$. So we fill the box above the combinations of TB with R.

![Third Row](walkthrough4_not.jpg)

5. Fill the final row with valid combinations of "top level" boxes.

The top level boxes are R, B, and {S, T}. None of the combinations (RB, BS, and BT) are valid so we are not able to fill anything in the next line.

![Final Row](walkthrough4_not.jpg)

6. Check if the top-left box contains the start symbol, S.

If the string is valid for the grammar, then the top-left box will contain the start symbol, S. Since our top-left box does not contain S, we can conclude that 'abab' is a not a valid string for the CFG, and therefore is not within the language defined by the CFG, a^nb^n for any n >= 0.

### Code Walkthrough

#### The `CYK_parse` Function

The `CYK_parse` function Parses a string using the CYK algorithm to determine if it can be derived from a given context-free grammar in Chomsky Normal Form (CNF). It takes a grammar (in the form of a dictionary) and a string, returning a tuple with a 2D parsing table and a boolean indicating if the string can be derived from the start symbol 'S'.

    def CYK_parse(grammar, string):

To begin with, the function calculates the length of the input string (`n`) and the number of rules in the grammar (`r`). 

    n = len(string)
    r = len(grammar)

It then creates a 2D list (`T`), where each empty cell is a set meant to store non-terminal symbols during parsing.

    T = [[set() for _ in range(n)] for _ in range(n)]

The parsing starts with substrings of length 1, which represent single characters of the input string.

    for i in range(n):

For each character in the string, the function iterates over the grammar's rules. If a character is part of a rule's right-hand side `rhs`, it implies that the corresponding left-hand side `lhs` non-terminal can produce this character. This LHS is then added to the corresponding cell in `T`.

        for lhs, rhs in grammar.items():
            if string[i] in rhs: 
                T[i][i].add(lhs)

Then this printed statement below displays which character from the string was found in the grammar rule and which non-terminal symbol was added to the table.

                print(f"{string[i]} found in {lhs, rhs}. Adding {lhs} to T[{i}][{i}]")

The table visualization is also updated for each iteration.

                print_table(T)



After handling single characters, the focus shifts to analyzing longer substrings (from 2 to `n`) of the input string. For each substring length `l`, and for each starting point `i`, it calculates the ending point `j`. 
     
     for l in range(2, n + 1):
        for i in range(n - l + 1):
            j = i + l - 1

It then iterates over all possible divisions of the substring (indexed by `k`) and checks if there's a production rule that can generate this substring. This is determined by checking if the LHS of a production rule can be formed by combining symbols found in smaller, already processed substrings of the current substring. 
            
            for k in range(i, j):
                for lhs, rhs in grammar.items():
The loop structure `for lhs, rhs in grammar.items():` goes through each rule in the grammar, where `lhs` represents a non-terminal symbol and `rhs` represents the production rules associated with that non-terminal. 

Within this loop, another nested loop iterates over each production rule (`prod`) in `rhs`.
                    
                    for prod in rhs:

To know if the non-terminal `lhs` can generate the substring spanning from `i` to `j`, these three conditions have to be met:

1. The production rule `prod` should be binary, meaning a length of 2, aligning with the CNF requirement that rules are either of the form A → BC or A → a.
2. `prod[0] in T[i][k]` ensures that the first non-terminal in the production rule can generate the substring at index `i` and ending at `k`.
3. `prod[1] in T[k + 1][j]` checks if the second non-terminal can derive the substring from index `k + 1` to `j`.
                    if len(prod) == 2 and prod[0] in T[i][k] and prod[1] in T[k + 1][j]:
This step is repeated for all possible substrings and their divisions to build the parsing table based on the grammar's production rules.

Then this printed statement below informs that the first symbol of the production rule (prod[0]) was found in the CYK table at position [i][k], and the second symbol (prod[1]) was found at [k+1][j]. The production rule being considered is thus valid, and will be added to the table

                        print(f"{prod[0]} found in T[{i}][{k}] and {prod[1]} found in T[{k+1}][{j}]. {lhs} : {prod[0]}{prod[1]} is valid. Adding {lhs} to T[{i}][{j}]")

If these conditions are met, then the non-terminal is added to the corresponding cell in table `T`.
                        
                        T[i][j].add(lhs)

The table visualization is also updated for each iteration.

                        print_table(T)

The full code implementation is below:

In [2]:
def CYK_parse(grammar, string):
    """
    Parses a string using the CYK algorithm to determine if it can be derived from a given context-free grammar in 
    Chomsky Normal Form (CNF).

    Args:
    grammar (dict): A dictionary representing the context-free grammar in CNF.
    string (str): The string to be parsed.

    Returns:
    tuple: A tuple consists of
        1. A 2D list representing the CYK parsing table, with each cell containing a set of non-terminal symbols.
        2. A boolean value indicating whether the string can be derived from the start symbol ('S').
    """

    n = len(string)
    r = len(grammar)
    
    # Create a 2D list table to store intermediate parsing results
    T = [[set() for _ in range(n)] for _ in range(n)]

    # The table is filled starting from substrings of length 1, gradually moving to longer substrings
    for i in range(n):
    # loops over the characters of the input string
        for lhs, rhs in grammar.items():
        # loops over the items in the grammar dictionary--
        # lhs:a non-terminal symbol; rhs: the set of production rules associated with the non-terminal symbol
            if string[i] in rhs:
            # if the ith character of the input string is part of the production rule set (rhs), 
            # the non-terminal symbol (lhs) can be used to derive this specific character according to the grammar rules
                T[i][i].add(lhs)
                # if the above is true, then non-terminal symbol (lhs) is added to the set at position [i][i] in the 2D list T
                print(f"{string[i]} found in {lhs, rhs}. Adding {lhs} to T[{i}][{i}]")
                # show character from the string was found in the grammar rule 
                # and which non-terminal symbol was added to the table
                print_table(T)
                # update the table visualization for each iteration

    # Fill in the table for substrings of length 2 to n
    for l in range(2, n + 1):
    # loops over the possible lengths of substrings, starting from 2 up to the length of the input string
        for i in range(n - l + 1):
        #starting from 0 to 
            j = i + l - 1
            # calculates the ending index (j) of the substring
            for k in range(i, j):
            # loops over the substring
                for lhs, rhs in grammar.items():
                # loops over the items in the grammar dictionary--
                # lhs:a non-terminal symbol; rhs: the set of terminal symbols (production rules) associated with the non-terminal.
                    for prod in rhs:
                    # iterates over each production rule "prod" in the set rhs.
                        if len(prod) == 2 and prod[0] in T[i][k] and prod[1] in T[k + 1][j]:
                        # if these 3 conditions are met:
                        # 1. the production rule prod is a binary
                        # 2. the first symbol of prod can generate the substring from i to k
                        # 3. the second symbol of prod can generate the substring from k + 1 to j
                            print(f"{prod[0]} found in T[{i}][{k}] and {prod[1]} found in T[{k+1}][{j}]. {lhs} : {prod[0]}{prod[1]} is valid. Adding {lhs} to T[{i}][{j}]")
                            # print out the progress
                            T[i][j].add(lhs)
                            # then add the non-terminal lhs, since it can generate the substring from i to j
                            print_table(T)
                            # update the table visualization for each iteration
                            
    # After the table is filled, the function checks if the start symbol ('S') is in the top-right cell of the table.
    print(f"Table filled. Checking if {T[0][n-1]} (T[0][{n-1}]) contains S")
    return T, 'S' in T[0][n - 1] 
    # the string can be derived from the start symbol according to the grammar if S (start) is an option in the top right corner

#### The `print_table` Function

The `print_table` function is designed to display the parsing table. This function accepts a 2D list (`table`) as its argument, where each cell in this list contains a set of symbols representing the results of the parsing process. 

    def print_table(table):

It first prints a header - "Parsing table:". Next, it engages in process to spin and drop the table. This does not affect the algorithm in any way, it just changes the way the table is printed so that it better matches human intuition and other examples in this text. First, it sets up values for n, the number of cells in the table as indexed from 0, and for two empty tables. The first empty table is used as a helper table to spin the values, while the second is used as the actual returned table. For the returned table, we set up "-" to fill the cells that are not actually used (i.e. the 'blacked out' cells.)

    n = len(table) - 1

    helper_table = [[" " for _ in range(n + 1)] for _ in range(n + 1)]
    new_table = [["-" for _ in range(n + 1)] for _ in range(n + 1)] #the remaining dashes in new_table after it is filled below represent boxes that are never used

Next, we engage in two loops through the tables. The first spins the table 90 degrees counterclockwise, as well as replaces any empty cells with " " instead of set(). By spinning the table, the value in (0, 0) (where (_, _) represents (row, column)) will fill the cell (4, 0), the value in (1, 0) will fill the cell (3, 0), etc.

    #spin table 90 deg
    for i, row in enumerate(table):
        for j, box in enumerate(row):
            if box != set(): # replace set() with " " as the empty symbol
                helper_table[(n - j)][i] = box

The second loop drops the values down. Before, the values are displayed from the top left corner to the diagonals, with any values below the diagonal (bottom right) as blacked out. By dropping the values down, teh first column stays the same, for the second column every value moves down by 1 cell, for the third column every value moves down by 2 cells, etc. Essentially what this is doing is making the diagonal into the bottom row and having the top right cells become the 'blacked out' cells.

    # drop values down so that diagonal becomes bottom row
    for i, row in enumerate(helper_table):
        for j, box in enumerate(row):
            if (j + i) <= n:
                new_table[i + j][j] = box


Lastly, with the new table, it prints each row, displaying the sets of symbols contained in each cell. These symbols represent the non-terminal symbols that could potentially derive the corresponding substrings of the input string as per the grammar rules used in the parsing algorithm.

        print("Parsing table:")
        for row in table:
            print(row)

The function is specifically designed for visualization, so it does not return any value but provides a clear representation of the current state of the parsing table, which helps with algoirthm explanation and verification.

The full code implementation is below:

In [3]:
def print_table(table):
    """
    Prints the parsing table.

    Args:
    table (list of list of sets): A 2D list representing a table where each cell contains a set of symbols.
                                  this is usually the parsing table generated from algorithms like CYK.

    Returns:
    - None: This function does not return values -- it solely prints and visualizes the table
    """
    print("Parsing table:")

    n = len(table) - 1

    helper_table = [[" " for _ in range(n + 1)] for _ in range(n + 1)]
    new_table = [["-" for _ in range(n + 1)] for _ in range(n + 1)] #the remaining dashes in new_table after it is filled below represent boxes that are never used


    #spin table 90 deg
    for i, row in enumerate(table):
        for j, box in enumerate(row):
            if box != set(): # replace set() with " " as the empty symbol
                helper_table[(n - j)][i] = box    
    
    # drop values down so that diagonal becomes bottom row
    for i, row in enumerate(helper_table):
        for j, box in enumerate(row):
            if (j + i) <= n:
                new_table[i + j][j] = box

    # display table
    for row in new_table:
        print(row)

#### Testing with Examples

The following code snippet presents an example of a grammar rule in CNF, along with sample strings. Strings that adhere to these grammar rules would return TRUE when input into the function, whereas strings that do not conform to these rules would yield a FALSE result after being processed by the function.

In [4]:
# Example grammar in CNF
# rules
grammar = {
    'S': {'AB'}, #S: start sym
    'A': {'BB', 'a'},
    'B': {'AB', 'b'}
}

# Example strings (TRUE)
string = "aabbb"
string1 = "aab"
string2 = "babbbb"
string3 = "abbbbaabbab"

# Example strings (FALSE)
string4 = ""
string5 = "b"
string6 = "abababababaa"


# Parse the string
table, result = CYK_parse(grammar, string1)

# Print the result
print("Can the string be derived?", result)

a found in ('A', {'BB', 'a'}). Adding A to T[0][0]
Parsing table:
[' ', '-', '-']
[' ', ' ', '-']
[{'A'}, ' ', ' ']
a found in ('A', {'BB', 'a'}). Adding A to T[1][1]
Parsing table:
[' ', '-', '-']
[' ', ' ', '-']
[{'A'}, {'A'}, ' ']
b found in ('B', {'b', 'AB'}). Adding B to T[2][2]
Parsing table:
[' ', '-', '-']
[' ', ' ', '-']
[{'A'}, {'A'}, {'B'}]
A found in T[1][1] and B found in T[2][2]. S : AB is valid. Adding S to T[1][2]
Parsing table:
[' ', '-', '-']
[' ', {'S'}, '-']
[{'A'}, {'A'}, {'B'}]
A found in T[1][1] and B found in T[2][2]. B : AB is valid. Adding B to T[1][2]
Parsing table:
[' ', '-', '-']
[' ', {'S', 'B'}, '-']
[{'A'}, {'A'}, {'B'}]
A found in T[0][0] and B found in T[1][2]. S : AB is valid. Adding S to T[0][2]
Parsing table:
[{'S'}, '-', '-']
[' ', {'S', 'B'}, '-']
[{'A'}, {'A'}, {'B'}]
A found in T[0][0] and B found in T[1][2]. B : AB is valid. Adding B to T[0][2]
Parsing table:
[{'S', 'B'}, '-', '-']
[' ', {'S', 'B'}, '-']
[{'A'}, {'A'}, {'B'}]
Table filled. Chec

### CYK Parsing on Natural Language

#### The Benefits of CYK in NLP: a Brief Comparative Analysis with Transformers
Natural Language Processing (NLP) is a field is an interdisciplinary subfield of computer science and linguistics. that is primarily concerned with giving computers the ability to support and manipulate human language (*Wikipedia*). It encompasses a range of algorithmic tools for enabling computers to understand and process human language. Among these tools are transformers and the CYK algorithm.

With the development and hype surrounding many large language models, transformers have garnered significant worldwide attention. While transformers are powerful tools in NLP, their approach to understanding language is based on statistical learning from large datasets. They are highly effective in many contexts *but might not always capture the syntactic structure of the language*. On the other hand, the CYK algorithm is another extreme. With its basis in formal grammar theory, it offers a strictly rule-based approach to parsing language. It intakes sentences that were broken down into their grammatical components, making CYK particularly valuable in applications such as in compiling programming languages or in certain aspects of natural language understanding, where parsing the exact structure of sentences is significant. 

Which brings us to the next topic: parts of speech.

#### What are Parts of Speech?

In NLP and other linguistics related domains, identifying parts of speech correctly is fundamental. Parts of speech refer to categories that group words with similar grammatical properties. Common categories in English include nouns, pronouns, verbs, adjectives, adverbs, prepositions, conjunctions, and so on. These categories are significant to define because words in the same part of speech typically adhere to similar grammatical rules.

Identifying parts of speech of different parts of a sentence is a preliminary step in the CYK algorithm, for CYK relies on a given grammar that is already in CNF to parse sentences. The grammar must match different words to parts of speech (like nouns, verbs, adjectives, etc.)like matching the single terminals to non-terminals, and define how the existing parts of speech (non-terminals) should be combined to form valid sentence structures, so that the algorithm can determine whether the input string (sentence) is structurally valid based on the grammar rules.


For example, consider a grammar in CNF that categorizes 'red' as an adjective, 'bird' as a noun, 'sings' as a verb, 'happily' as an adverb, 'in' as a preposition, 'the' as a definite article, and 'spring' as a noun. The CYK algorithm, using this grammar, can analyze a sentence like 'The red bird sings happily.'

#### Writing CNF CFGs for Natural Language Parsing

Writing a Context-Free Grammar (CFG) in Chomsky Normal Form (CNF) for parsing natural language sentences involves restructuring complex linguistic rules into a binary format. For instance, consider the simple sentence "The cat sleeps." A basic CFG to parse this might look like
$$S \rightarrow (NP)(VP)$$
$$NP \rightarrow (Det)(Noun)$$
$$VP \rightarrow Verb$$
$$Det \rightarrow the$$
$$Noun \rightarrow cat$$
$$Verb \rightarrow sleeps$$

where NP, VP, Det, Noun, and Verb are are non-terminals and the, cat, and sleeps are all terminals.

The abbreviations represent grammatical categories or parts of speech, where:

**NP** stands for "Noun Phrase." A noun phrase is a grammatical unit that can include a noun along with accompanying modifiers, such as articles, adjectives, pronouns, or other nouns. In the sentence "The cat sleeps," "The cat" is an example of a noun phrase.

**VP** represents "Verb Phrase." A verb phrase consists of a main verb and, optionally, an object, and other modifiers. It can express actions, events, or states of being. In "The cat sleeps," "sleeps" forms a simple verb phrase.

**Det** is short for "Determiner." Determiners are words that come before nouns and provide context in terms of definiteness, proximity, quantity, or ownership. Examples include "the," "a," "this," "those," "many," etc. In the example sentence, "The" is a determiner.

**Noun** is a word that identifies a person, place, thing, or idea. Nouns can function as the subject or object of a verb. In "The cat sleeps," "cat" is the noun.

**Verb** refers to a word that describes an action, state, or occurrence. Verbs form the main part of the predicate of a sentence. In the example, "sleeps" is the verb.

Notice that our grammar is not yet in Chomsky Normal Form. We can convert it to CNF by applying the process described in *Converting CFGs into Chomsky Normal Form (CNF)*.

#### Parsing a Sentence (Visual Walkthrough)
Let's parse the sentence "the cat sleeps". 

1. We've already written our CFG for this sentence. As a reminder, the CFG is:
$$S \rightarrow (NP)(VP)$$
$$NP \rightarrow (Det)(Noun)$$
$$VP \rightarrow Verb$$
$$Det \rightarrow the$$
$$Noun \rightarrow cat$$
$$Verb \rightarrow sleeps$$

2. Our CFG above is not quite in CNF. Let's convert it to CNF by applying the steps from *Converting CFGs into Chomsky Normal Form (CNF)*. One possible CNF CFG for this sentence is:

$$S \rightarrow (NP)(VP)$$
$$NP \rightarrow (Det)(Noun)$$
$$VP \rightarrow sleeps$$
$$Det \rightarrow the$$
$$Noun \rightarrow cat$$

3. Using the CFG rules, we can build a parse tree for our sentence.

To build the parse tree, we go through the rules in our grammar and add appropriate nodes or leaves to the tree. Nodes represent non-terminals and leaves represent terminals. We can start our tree with the root S. Following the first rule in our grammar, $S \rightarrow (NP)(VP)$, we can add two nodes, NP and VP, off of S. Following the second rule, $NP \rightarrow (Det)(Noun)$, we can add two nodes off of NP, Det and Noun. Lastly we can add our three leaves following the rules $VP \rightarrow sleeps$, $Det \rightarrow the$, and $Noun \rightarrow cat$. The grey Verb node represents that the rule $VP \rightarrow sleeps$ moves through the Verb non-terminal. It is included in grey here for clarity, but is not necessary for the tree.

!['the cat sleeps' Parse Tree](parsetree_thecatsleeps.jpg)

4. Using the CFG rules, we can build a chart for our sentence.

To build the chart, we start with the terminals and build backwards using the rules of the grammar. If it helps, you can think of it as building our parse tree in reverse. The terminal 'the' goes to the non-terminal 'Det'. The terminal 'cat' goes to the non-terminal 'Noun'. The terminal 'sleeps' goes to the non-terminal 'VP'. On the next line, we look to see if there are any valid combinations of non-terminals. We can find that there is one valid combination; the non-terminals 'Det' and 'Noun' go back to the non-terminal 'NP'. There are no other valid combinations on this line so we move up to the next line. On the top line, we find the valid combination 'NP VP' which goes to 'S'. 

!['the cat sleeps' Chart](chart_thecatsleeps.jpg)

5. Check if 'S' is present in the top left box on the chart.

Once we complete our chart, we can check if 'S' is present in the top left box. If it is, then the string is valid for the grammar since that shows that we can get to the string we want starting from 'S' and going through a process of valid rules. Note that S could be present in other places on the chart, but we are only looking for its presence in the top left box since that box represents the highest level of rules, aka the start. Since our chart has 'S' present in the top left corner, the string "the cat sleeps" is valid for the given grammar.


### Appendix

#### Instructions for Running the Algorithm

To run the CYK Algorithm implementation, fill in the lines 'grammar = ' and 'string = ' with any grammar and associated strings from the example section, then run the code chunk using the triangle shaped run button the top left of the code chunk. To create your own grammar, follow the rules in *Converting CFGs into Chomsky Normal Form (CNF)* to write a CNF grammar for a language of your choice.

#### CYK Implementation with Examples

To see the full algorithm, check out the *Code Walkthrough* section or view the file "CYKalgorithm.py".

In [26]:
#from CYKalgorithm import CYK_parse

## Examples

# Grammar 1
palindrome_grammar = {
    'S': {'AY', 'BZ', '','a','b', 'AA', 'BB'},
    'Y': {'SA'},
'Z': {'SB'},
'A': {'a'},
'B': {'b'},
}

# Strings (TRUE)
string_palindrome = "aba"
string1_palindrome = ""
string2_palindrome = "bababbabab"

# Strings (FALSE)
string3_palindrome = "aabb"
string4_palindrome = "ab"
string5_palindrome = "baabab"


# Grammar 2
CFG_anbm_grammar = {
    'S': {'AB', 'A'},
    'A': {'XA', 'a'},
    'B': {'YB', 'b'},
    'X': {'a'}, 
    'Y': {'b'}   
}

# Strings (TRUE)
string_CFG= "ab"
string1_CFG = "aaaaaaabbb"
string2_CFG = "abbb"

# Strings (FALSE)
string3_CFG = "aababb"
string4_CFG = "abbababa"
string5_CFG = "aa"


## Create your own! 

# Create your own grammar

grammar_diy = {

}

# Create your own strings (TRUE)
string_diy = ""
string_diy_1 = ""

# Create your own strings (FALSE)
string_diy_3 = ""
string_diy_4 = ""

## Parsing

# Parse the string
grammar = CFG_anbm_grammar #the name of example grammar you want to run
string = string_CFG  #the name of the example string you want to run (or type in your own string in the format "your string")

table, result = CYK_parse(grammar, string)

# Print the result
print("Can the string be derived?", result)

a found in ('A', {'XA', 'a'}). Adding A to T[0][0]
Parsing table:
[' ', '-', '-', '-', '-', '-', '-', '-', '-', '-']
[' ', ' ', '-', '-', '-', '-', '-', '-', '-', '-']
[' ', ' ', ' ', '-', '-', '-', '-', '-', '-', '-']
[' ', ' ', ' ', ' ', '-', '-', '-', '-', '-', '-']
[' ', ' ', ' ', ' ', ' ', '-', '-', '-', '-', '-']
[' ', ' ', ' ', ' ', ' ', ' ', '-', '-', '-', '-']
[' ', ' ', ' ', ' ', ' ', ' ', ' ', '-', '-', '-']
[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '-', '-']
[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '-']
[{'A'}, ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
a found in ('X', {'a'}). Adding X to T[0][0]
Parsing table:
[' ', '-', '-', '-', '-', '-', '-', '-', '-', '-']
[' ', ' ', '-', '-', '-', '-', '-', '-', '-', '-']
[' ', ' ', ' ', '-', '-', '-', '-', '-', '-', '-']
[' ', ' ', ' ', ' ', '-', '-', '-', '-', '-', '-']
[' ', ' ', ' ', ' ', ' ', '-', '-', '-', '-', '-']
[' ', ' ', ' ', ' ', ' ', ' ', '-', '-', '-', '-']
[' ', ' ', ' ', ' ', ' ', ' ', ' ', '-', '-', '-']
[' ',

### Sources
“Automata Chomsky’s Normal Form (CNF) - Javatpoint.” Www.Javatpoint.Com, www.javatpoint.com/automata-chomskys-normal-form. Accessed 14 Dec. 2023.

“Chomsky Normal Form.” Online Tutorials, Courses, and eBooks Library, www.tutorialspoint.com/automata_theory/chomsky_normal_form.htm. Accessed 14 Dec. 2023. 

“Cocke–Younger–Kasami (CYK) Algorithm.” GeeksforGeeks, GeeksforGeeks, 13 Feb. 2023, www.geeksforgeeks.org/cocke-younger-kasami-cyk-algorithm/. 

“Context-Free Grammar.” Wikipedia, Wikimedia Foundation, 13 Dec. 2023, en.wikipedia.org/wiki/Context-free_grammar. 

“Converting Context Free Grammar to Chomsky Normal Form.” GeeksforGeeks, GeeksforGeeks, 21 May 2019, www.geeksforgeeks.org/converting-context-free-grammar-chomsky-normal-form/. 

“CYK Algorithm for Context Free Grammar.” GeeksforGeeks, GeeksforGeeks, 22 June 2020, www.geeksforgeeks.org/cyk-algorithm-for-context-free-grammar/. 

“Cyk Algorithm.” Wikipedia, Wikimedia Foundation, 27 Nov. 2023, en.wikipedia.org/wiki/CYK_algorithm. 

“Educative Answers - Trusted Answers to Developer Questions.” Educative, www.educative.io/answers/what-is-the-cyk-algorithm. Accessed 14 Dec. 2023. 

“Natural Language Processing.” Wikipedia, Wikimedia Foundation, 28 Nov. 2023, en.wikipedia.org/wiki/Natural_language_processing. 

“Tutorial #15: Parsing I Context-Free Grammars and the CYK Algorithm.” Borealis AI, 8 June 2023, www.borealisai.com/research-blogs/tutorial-15-parsing-i-context-free-grammars-and-cyk-algorithm/.